# Import libraries

In [24]:
# import necessary libraries
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
sns.set()

# Obtain client_email

In [25]:
def client_email(creds_file):
    # read and open creds.json file
    with open(creds_file) as f:
        creds = json.load(f)
    # get client_email 
    client_email=creds['client_email'] 
    return client_email

# Create a function to extract data from a specific google sheet

In [10]:
# obtain the client email and the data from google sheet with dataframe type
def gs_to_df(creds_file,sheet_name):
    # Read Data from a Spreadsheet
    gc = gspread.service_account(filename=creds_file) # gspread – to interact with Google Spreadsheets
    # extract data from google sheet by the name of the sheet
    sheet = gc.open(sheet_name) 
    #For the first sheet, pass the index 0 and so on.
    sheet_instance = sheet.get_worksheet(0) 
    # get all the records of the data
    records_data = sheet_instance.get_all_records()
    # convert the json to dataframe
    df = pd.DataFrame(records_data)
    return df, sheet

# Create a function to upload an updated data to your Google Sheets on a new sheet

In [7]:
def upload_data_to_gs(title,sheet_page,updated_df):
    row=updated_df.shape[0]
    col=updated_df.shape[1]
    # add a sheet with 20 rows and 2 columns
    sheet.add_worksheet(rows=row,cols=col,title=title)
    # get the instance of the second sheet
    new_sheet = sheet.get_worksheet(sheet_page)
    # Update values to the sheet
    new_sheet.insert_rows(updated_df.values.tolist())
    # add headers
    header = list(updated_df.columns)
    index = 1
    new_sheet.insert_row(header, index)

#  Extract data from a google sheet

In [11]:
# Get client_email and share the Google Sheet with this email
client_email=client_email('creds.json')
# After share the google sheet with the client email then let us extract the data 
df, sheet=gs_to_df('creds.json',"Consumer_Behavior_Data")

In [13]:
df.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Timestamp,Year,Month,Day,Hour,Gender,Clicked on Ad,Age Range
0,68.95,35,"$61,833.90",256.09,Cloned 5thgeneration orchestration,Wrightburgh,Tunisia,2016-03-27 0:53:11,2016,3,27,0,Female,Clicked,26-35
1,80.23,31,"$68,441.85",193.77,Monitored national standardization,West Jodi,Nauru,2016-04-04 1:39:02,2016,4,4,1,Male,Clicked,26-35
2,69.47,26,"$59,785.94",236.50,Organic bottom-line service-desk,Davidton,San Marino,2016-03-13 20:35:42,2016,3,13,20,Female,Clicked,26-35
3,74.15,29,"$54,806.18",245.89,Triple-buffered reciprocal time-frame,West Terrifurt,Italy,2016-01-10 2:31:19,2016,1,10,2,Male,Clicked,26-35
4,68.37,35,"$73,889.99",225.58,Robust logistical utilization,South Manuel,Iceland,2016-06-03 3:36:18,2016,6,3,3,Female,Clicked,26-35


# Let us create a new data

In [22]:
new_data=df.groupby(['Clicked on Ad','Gender'])["Daily Internet Usage"].mean().reset_index()
new_data

,Clicked on Ad,Gender,Daily Internet Usage
0,Clicked,Female,214.623440
1,Clicked,Male,214.404040
2,Not Clicked,Female,145.539182
3,Not Clicked,Male,145.425065


# Now let us upload this new_data to the GS with on a new sheet 

In [23]:
upload_data_to_gs('target_gender_DailyInternetUsage',4,new_data)